# **TP : Étude des Cartes de Contrôle CUSUM**

### **Auteur : KHELID Lilya-Nada**

---

## **Objectifs :**  
Dans ce notebook, nous allons explorer l’utilisation des **cartes de contrôle CUSUM** en contrôle statistique des procédés (CSP). L’objectif est de comprendre et d’implémenter cette méthode permettant de détecter rapidement les dérives d’un processus en surveillant l’accumulation des écarts par rapport à une valeur cible.  

Les thématiques principales abordées sont :  

1. **Visualisation et interprétation des cartes CUSUM.**  
2. **Construction des cartes CUSUM pour le suivi des procédés.**  
3. **Analyse des seuils de décision et détection précoce des anomalies.**  
4. **Comparaison de la sensibilité des cartes CUSUM avec les cartes de Shewhart.**

Ce notebook implémente les différentes cartes de contrôle demandées dans le TP, y compris CUSUM, EWMA et Moyenne Mobile.

---

### 📥 Importation des librairies

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats

In [6]:
#--Exercice 1: Carte CUSUM--

def cusum(data, mu0, sigma0, k, h):
    cusum_pos = np.zeros(len(data))
    cusum_neg = np.zeros(len(data))
    for i in range(1, len(data)):
        cusum_pos[i] = max(0, data[i] - (mu0 + k) + cusum_pos[i-1])
        cusum_neg[i] = max(0, (mu0 - k) - data[i] + cusum_neg[i-1])
    return cusum_pos, cusum_neg

#--Exercice 2: Carte EWMA--

def ewma(data, lambda_, L, mu0, sigma0):
    ewma_values = np.zeros(len(data))
    ewma_values[0] = mu0
    for i in range(1, len(data)):
        ewma_values[i] = lambda_ * data[i] + (1 - lambda_) * ewma_values[i-1]
    ucl = mu0 + L * sigma0 * np.sqrt(lambda_ / (2 - lambda_))
    lcl = mu0 - L * sigma0 * np.sqrt(lambda_ / (2 - lambda_))
    return ewma_values, ucl, lcl

#--Exercice 3: Moyenne Mobile--

def moving_average(data, w):
    return np.convolve(data, np.ones(w)/w, mode='valid')

Preparons nos plots :

In [7]:
def plot_cusum(data, cusum_pos, cusum_neg, mu0):
    df = pd.DataFrame({'Index': np.arange(len(data)), 'CUSUM Positif': cusum_pos, 'CUSUM Négatif': cusum_neg})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Index'], y=df['CUSUM Positif'], mode='lines', name='CUSUM Positif'))
    fig.add_trace(go.Scatter(x=df['Index'], y=df['CUSUM Négatif'], mode='lines', name='CUSUM Négatif'))
    fig.add_trace(go.Scatter(x=df['Index'], y=[mu0]*len(df), mode='lines', name='Moyenne sous contrôle', line=dict(dash='dash')))
    fig.update_layout(title='Carte CUSUM')
    fig.show()

def plot_ewma(data, ewma_values, ucl, lcl):
    df = pd.DataFrame({'Index': np.arange(len(data)), 'EWMA': ewma_values})
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df['Index'], y=df['EWMA'], mode='lines', name='EWMA', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=df['Index'], y=[ucl]*len(df), mode='lines', name='UCL', line=dict(dash='dash')))
    fig.add_trace(go.Scatter(x=df['Index'], y=[lcl]*len(df), mode='lines', name='LCL', line=dict(dash='dash')))
    fig.update_layout(title='Carte EWMA')
    fig.show()

def plot_moving_avg(moving_avg):
    df = pd.DataFrame({'Index': np.arange(len(moving_avg)), 'Moyenne Mobile': moving_avg})
    fig = px.line(df, x='Index', y='Moyenne Mobile', title='Carte Moyenne Mobile')
    fig.show()

In [8]:
# Prenons :
data = np.random.normal(1050, 25, 100)
mu0, sigma0 = 1050, 25
k, h = 0.5, 4.77
lambda_, L = 0.10, 2.7
w = 6

cusum_pos, cusum_neg = cusum(data, mu0, sigma0, k, h)
ewma_values, ucl, lcl = ewma(data, lambda_, L, mu0, sigma0)
moving_avg = moving_average(data, w)

-----

In [9]:
plot_cusum(data, cusum_pos, cusum_neg, mu0)

plot_ewma(data, ewma_values, ucl, lcl)

plot_moving_avg(moving_avg)
